In [33]:
import cv2                 
import numpy as np         
import os 
from random import shuffle

In [34]:
# CTE: These are my filepaths, here for convenience
cte_train_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\train\\'
cte_test_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\test\\'
cte_additional_t1_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\additional_Type_1_v2\\'
cte_additional_t2_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\additional_Type_2_v2\\'
cte_additional_t3_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\additional_Type_3_v2\\'

hjh_train_dir = '/Users/hayleeheilman/Data/Train'
hjh_test_dir = '/Users/hayleeheilman/Data/test'
hjh_additional_t1_dir = '/Users/hayleeheilman/Data/additional_Type_1'
hjh_additional_t2_dir = '/Users/hayleeheilman/Data/additional_Type_2'
hjh_additional_t3_dir = '/Users/hayleeheilman/Data/additional_Type_3'

TRAIN_DIR = cte_train_dir # put the path to the training data here.
TEST_DIR = cte_test_dir # put the path to the test data here.
IMG_SIZE = 50 # we can change this to whatever
STORAGE_DIR = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\processed\\'

In [35]:
# Progress bar function, since some of these processes take a while
def show_progress(current_index, total_len):
    x = (current_index + 1) * 100 // total_len
    x_less1 = current_index * 100 // total_len
    if (x % 10 == 0 and x - x_less1 > 0 and x < 100):
        print('{}%...'.format(x), end='')
    elif current_index == total_len - 1:
        print('100%')

In [36]:
def label_img(file):
    data_label = file
    # conversion to one-hot array [type_1,type_2,type_3]
    if data_label == 'Type_1': return [1,0,0]
    elif data_label == 'Type_2': return [0,1,0]
    elif data_label == 'Type_3': return [0,0,1]

In [37]:
def create_train_data(train_dir):
    training_data = []
    for file in os.listdir(train_dir):
        print('Starting on file {}'.format(file))
        file_path = os.path.join(train_dir,file)
        data_label = label_img(file)
        file_len = len(os.listdir(file_path))
        for i, img in enumerate(os.listdir(file_path)):
            show_progress(i,file_len) # Print progress
            label = data_label
            path = os.path.join(file_path,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [38]:
def process_test_data():
    testing_data = []
    file_len = len(os.listdir(TEST_DIR))
    for i, img in enumerate(os.listdir(TEST_DIR)):
        show_progress(i,file_len) # Print progress
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

In [40]:
train_folders = [hjh_train_dir]
for folder in train_folders:
    train_data = create_train_data(folder)
#process_test_data()

Starting on file Type_1
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%
Starting on file Type_2
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%
Starting on file Type_3
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


In [41]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [65]:
batch_size = 25
num_classes = 3
epochs = 20
train = train_data[:-300]
test = train_data[-300:]
x_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = np.array([i[1] for i in train])
x_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_test = np.array([i[1] for i in test])

In [66]:
model = Sequential()
model.add(Conv2D(32,(3,3),
                activation='relu',
                input_shape=(50,50,1)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [67]:
model.fit(x_train, y_train,
         batch_size=25,
         epochs=20,
         verbose=1,
         validation_data=(x_test, y_test))

Train on 1181 samples, validate on 300 samples
Epoch 1/20
1181/1181 [==============================] - 12s - loss: 1.1676 - acc: 0.4767 - val_loss: 1.0300 - val_acc: 0.5067